In [108]:
import sqlite3
import pandas as pd

In [109]:
con = sqlite3.connect(':memory:')

In [110]:
table_names = [
    'Categories',   # категории товаров
    'Customers',    # заказчики
    'Employees',    # сотрудники
    'OrderDetails', # детали заказов
    'Orders',       # заказы
    'Products',     # товары
    'Shippers',     # перевозчики
    'Suppliers',    # поставщики
]

In [111]:
for table in table_names:
    data = pd.read_csv(f'{table}.csv', sep = ';')
    data.to_sql(table, con, index=False, if_exists='replace')

In [112]:
tasks = 0

### Все задания нужно выполнить только с помощью sql

1. Выберите заказчиков из Германии, Франции и Мадрида, выведите их название, страну и адрес.

In [113]:
tasks+=1
pd.read_sql(
    '''
    SELECT Distinct (CustomerName), Country, Address
    FROM Customers
    WHERE Country in ('Germany', 'France')
    or City = 'Madrid'
    ''',
    con,
)

,CustomerName,Country,Address
0,Alfreds Futterkiste,Germany,Obere Str. 57
1,Blauer See Delikatessen,Germany,Forsterstr. 57
2,Blondel père et fils,France,"24, place Kléber"
3,Bólido Comidas preparadas,Spain,"C/ Araquil, 67"
4,Bon app',France,"12, rue des Bouchers"
5,Drachenblut Delikatessend,Germany,Walserweg 21
6,Du monde entier,France,"67, rue des Cinquante Otages"
7,FISSA Fabrica Inter. Salchichas S.A.,Spain,"C/ Moralzarzal, 86"
8,Folies gourmandes,France,"184, chaussée de Tournai"
9,Frankenversand,Germany,Berliner Platz 43


2. Выберите топ 3 страны по количеству заказчиков, выведите их названия и количество записей.

In [114]:
tasks+=1
pd.read_sql(
    '''
    SELECT 
        CustomerName,
        COUNT(CustomerName) AS Number_of_customers
    FROM Customers
    GROUP BY Country
    ORDER BY Number_of_customers DESC 
    LIMIT 3
    ''',
    con,
)

,CustomerName,Number_of_customers
0,Great Lakes Food Market,13
1,Alfreds Futterkiste,11
2,Blondel père et fils,11


3. Выберите перевозчика, который отправил 10-й по времени заказ, выведите его название, и дату отправления.

In [115]:
tasks+=1
pd.read_sql(
    '''
    SELECT ShipperName, OrderDate
    FROM Shippers AS Sh JOIN Orders AS Ord
    ON Sh.ShipperID == Ord.ShipperID
    Order by OrderDate
    LIMIT 9,1
    ''',
    con,
)

,ShipperName,OrderDate
0,Federal Shipping,02.10.1996


4. Выберите самый дорогой заказ, выведите список товаров с их ценами.

In [116]:
tasks+=1
pd.read_sql(
    '''
    WITH find_total_price_for_quantity as (
        SELECT 
            Od.OrderID,
            Od.ProductID,
            Pr.ProductName,
            sum(Od.Quantity)*Pr.Price as Cost_for_quantity

        FROM OrderDetails AS Od JOIN Products AS Pr
        ON Od.ProductID = Pr.ProductID
    
        group by Od.OrderID, Od.ProductID, Pr.ProductName
    ),
    find_total_cost_for_order as (
        SELECT 
            OrderID,
            SUM(Cost_for_quantity) AS Total_cost_for_order
        FROM find_total_price_for_quantity
        GROUP BY OrderID
        ORDER BY Total_cost_for_order DESC
        LIMIT 1
    )
    SELECT 
        OrderID,
        ProductName,
        Cost_for_quantity
    FROM find_total_price_for_quantity
    WHERE OrderID = (SELECT OrderID FROM find_total_cost_for_order)
    ORDER BY Cost_for_quantity
        
    ''',
    con,
)

,OrderID,ProductName,Cost_for_quantity
0,10372,Sir Rodney's Marmalade,972.0
1,10372,Mozzarella di Giovanni,1461.6
2,10372,Camembert Pierrot,2380.0
3,10372,Côte de Blaye,10540.0


5. Какой товар больше всего заказывали по количеству единиц товара, выведите его название и количество единиц в каждом из заказов.

In [117]:
tasks+=1
pd.read_sql(
    '''
    SELECT 
        ProductName,
        Quantity,
        OrderID
    FROM OrderDetails AS Od JOIN Products AS Pr
        ON Od.ProductID = Pr.ProductID
    WHERE Od.ProductID == (
        SELECT 
            ProductID
        FROM OrderDetails
        GROUP BY OrderID, ProductID
        ORDER BY SUM(Quantity) DESC, OrderID
        LIMIT 1
    )
    ORDER BY Quantity DESC

    
    ''',
    con,
)

,ProductName,Quantity,OrderID
0,Pâté chinois,120,10398
1,Pâté chinois,40,10342
2,Pâté chinois,21,10254
3,Pâté chinois,20,10280
4,Pâté chinois,15,10389
5,Pâté chinois,12,10356
6,Pâté chinois,10,10425


6. Выведите топ 5 поставщиков по количеству заказов, выведите их названия, страну, контактное лицо и телефон.

In [118]:
tasks+=1
pd.read_sql(
    '''
    SELECT 
        SupplierName, 
        Country,
        ContactName,
        Phone
    FROM Suppliers
    WHERE SupplierID IN (
        SELECT 
            SupplierID
        FROM OrderDetails AS Ord JOIN Products AS Pr
        ON Ord.ProductID = Pr.ProductID
        GROUP BY SupplierID
        ORDER BY COUNT(OrderID) DESC
        LIMIT 5
    )
    ''',
    con,
)

,SupplierName,Country,ContactName,Phone
0,"Pavlova, Ltd.",Australia,Ian Devling,(03) 444-2343
1,"Specialty Biscuits, Ltd.",UK,Peter Wilson,(161) 555-4448
2,Plutzer Lebensmittelgroßmärkte AG,Germany,Martin Bein,(069) 992755
3,Formaggi Fortini s.r.l.,Italy,Elio Rossi,(0544) 60323
4,Norske Meierier,Norway,Beate Vileid,(0)2-953010


7. Какую категорию товаров заказывали больше всего по стоимости в Бразилии, выведите страну, название категории и сумму.

In [119]:
tasks+=1
pd.read_sql(
    '''
    WITH Main_table AS ( 
        SELECT 
            Od.OrderID,
            Pr.ProductID, 
            Cat.CategoryName,
            Cus.Country,
            sum(Od.Quantity)*Pr.Price as Cost_for_quantity

        FROM OrderDetails AS Od 

            JOIN Products AS Pr ON Od.ProductID = Pr.ProductID
            JOIN Categories AS Cat ON Cat.CategoryID = Pr.CategoryID
            JOIN Orders AS Ord ON Od.OrderID = Ord.OrderID
            JOIN Customers AS Cus ON Cus.CustomerID = Ord.CustomerID

        WHERE Cus.Country = 'Brazil'
        GROUP BY Od.OrderID, Od.ProductID, Cat.CategoryName, Cus.Country
    )
    SELECT 
        Country,
        CategoryName,
        sum(Cost_for_quantity) as Cost_for_category
    FROM Main_table
    GROUP BY CategoryName, Country
    ORDER BY Cost_for_category DESC
    LIMIT 1
    ''',
    con,
)

,Country,CategoryName,Cost_for_category
0,Brazil,Beverages,13690.0


8. Какая разница в стоимости между самым дорогим и самым дешевым заказом из США.

In [120]:
tasks+=1
pd.read_sql(
    '''
    WITH find_total_price_for_quantity as (
        SELECT 
            Od.OrderID,
            Od.ProductID,
            sum(Od.Quantity)*Pr.Price as Cost_for_quantity

        FROM OrderDetails AS Od JOIN Products AS Pr
        ON Od.ProductID = Pr.ProductID
    
        group by Od.OrderID, Od.ProductID, Pr.ProductName
    ),
    Total_cost_for_order_for_USA as (
        SELECT 
            SUM(Cost_for_quantity) AS Total_cost_for_order
        FROM find_total_price_for_quantity AS TP 
            JOIN Orders AS Ord ON Tp.OrderID = Ord.OrderID
            JOIN Customers AS Cus ON Cus.CustomerID = Ord.CustomerID
        WHERE Cus.Country = 'USA'
        GROUP BY Ord.OrderID
    )
    SELECT 
        MAX(Total_cost_for_order) - MIN(Total_cost_for_order) AS Diff_between_max_and_min_cost
    FROM Total_cost_for_order_for_USA  
    ''',
    con,
)

,Diff_between_max_and_min_cost
0,7638.45


9. Выведите количество заказов у каждого их трех самых молодых сотрудников, а также имя и фамилию во второй колонке.

In [121]:
tasks+=1
pd.read_sql(
    '''
    SELECT
        COUNT(OrderID) AS Count_of_orders,
        Emp.FirstName || ' '|| Emp.LastName AS Name_and_surname
   
    FROM Orders AS Ord JOIN Employees AS Emp ON Emp.EmployeeID = Ord.EmployeeID
    GROUP BY Emp.EmployeeID
    ORDER BY substr(BirthDate,7,4) DESC, substr(BirthDate,4,2) DESC, substr(BirthDate,1,2) DESC
    LIMIT 3
    ''',
    con,
)

,Count_of_orders,Name_and_surname
0,6,Anne Dodsworth
1,29,Nancy Davolio
2,31,Janet Leverling


10. Сколько банок крабового мяса всего было заказано.

In [122]:
tasks+=1
pd.read_sql(
    '''
    SELECT 
        SUM(Ord.Quantity * substr(Pr.Unit,1,2)) AS Total_number_of_tins
    FROM Products AS Pr Join OrderDetails AS Ord ON Pr.ProductID=Ord.ProductID
    WHERE ProductName like 'Boston Crab Meat'
    ''',
    con,
)

,Total_number_of_tins
0,6144


In [123]:
if tasks==10:
    print('Выполнены все задания 🙂')
else:
    print('Выполнены не все задания! 🙀')

Выполнены все задания 🙂
